In [1]:
import os

os.environ['WANDB_MODE'] = 'disabled'
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_HOME"] = "/usr/local/cuda-12.4"

In [2]:
!pip install -U bitsandbytes peft accelerate datasets sentencepiece wandb python-dotenv wtpsplit
!pip install flash-attn --no-build-isolation
!pip install wtpsplit==2.1.1
!pip install syntok==1.4.4
!pip install omegaconf
!pip install wandb
!pip install --upgrade transformers trl
!pip install pandas numpy

  Using cached wtpsplit-2.1.4-py3-none-any.whl.metadata (746 bytes)
  Using cached transformers-4.47.1-py3-none-any.whl.metadata (44 kB)
Using cached wtpsplit-2.1.4-py3-none-any.whl (124 kB)
Using cached transformers-4.47.1-py3-none-any.whl (10.1 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.2
    Uninstalling transformers-4.48.2:
      Successfully uninstalled transformers-4.48.2
  Attempting uninstall: wtpsplit
    Found existing installation: wtpsplit 2.1.1
    Uninstalling wtpsplit-2.1.1:
      Successfully uninstalled wtpsplit-2.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vllm 0.7.1 requires torch==2.5.1, but you have torch 2.6.0 which is incompatible.
vllm 0.7.1 requires transformers>=4.48.2, but you have transformers 4.47.1 which is incompatible.
  Using cached wtpsplit-2.1.1-py3-none-any.whl.metadata

In [8]:
!pip uninstall torch torchvision -y
!pip install torch torchvision

Found existing installation: torch 2.6.0
Uninstalling torch-2.6.0:
  Successfully uninstalled torch-2.6.0
Found existing installation: torchvision 0.20.1+cu124
Uninstalling torchvision-0.20.1+cu124:
  Successfully uninstalled torchvision-0.20.1+cu124
  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl (766.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 111.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.28.post3 requires torch==2.5.1, but you have torch 2.6.0 which is incompatible.
vllm 0.7.1 requires torch==2.5.1, but you have torch 2.6.0 which is incompatible.
vllm 0.7.1 requires torchvision==0.20.1, but you have torchvision 0.21.0 which is incompatible.
torchaudio 2.5.1+cu124 requires torch==2.5.1, but you have torch 2.6.0 which 

In [1]:
import os

from omegaconf import OmegaConf
from dotenv import load_dotenv
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from huggingface_hub import login
import argparse
import logging
import torch
import wandb
from typing import List
from transformers import PreTrainedTokenizerBase, BitsAndBytesConfig
from tqdm import tqdm
from torch.utils.data import Dataset
from transformers import PreTrainedTokenizerBase
import pickle
from trl.trainer import ConstantLengthDataset
import pandas as pd
from datasets import Dataset
from transformers.trainer_callback import EarlyStoppingCallback

In [2]:
hf_key   = "REDACTED"
secret_wandb = "REDACTED"

In [3]:
wandb_project_name = f'NLP-LAB-3-GENAI'

wandb.login(key = secret_wandb)
login(hf_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rkovalch (rkovalchuk) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
import gc

def clear_gpu_memory():
    torch.cuda.empty_cache()
    print(gc.collect())
clear_gpu_memory()

90


In [5]:
!env TORCH_USE_CUDA_DSA=1

SHELL=/bin/bash
NV_LIBCUBLAS_VERSION=12.5.3.2-1
NVIDIA_VISIBLE_DEVICES=all
COLAB_JUPYTER_TRANSPORT=ipc
NV_NVML_DEV_VERSION=12.5.82-1
NV_CUDNN_PACKAGE_NAME=libcudnn9-cuda-12
CGROUP_MEMORY_EVENTS=/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events
NV_LIBNCCL_DEV_PACKAGE=libnccl-dev=2.22.3-1+cuda12.5
NV_LIBNCCL_DEV_PACKAGE_VERSION=2.22.3-1
VM_GCE_METADATA_HOST=169.254.169.253
HOSTNAME=2060f06fc12b
LANGUAGE=en_US
TBE_RUNTIME_ADDR=172.28.0.1:8011
COLAB_TPU_1VM=
GCE_METADATA_TIMEOUT=3
NVIDIA_REQUIRE_CUDA=cuda>=12.5 brand=unknown,driver>=470,driver<471 brand=grid,driver>=470,driver<471 brand=tesla,driver>=470,driver<471 brand=nvidia,driver>=470,driver<471 brand=quadro,driver>=470,driver<471 brand=quadrortx,driver>=470,driver<471 brand=nvidiartx,driver>=470,driver<471 brand=vapps,driver>=470,driver<471 brand=vpc,driver>=470,driver<471 brand=vcs,driver>=470,driver<471 brand=vws,driver>=470,driver<471 brand=cloudgaming,driver>=470,driver<471 brand=unknown,driver>=535,dr

In [6]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [7]:
train_df = pd.read_json("/gdrive/MyDrive/genainlp3/zno.train.jsonl", lines=True)
test_df = pd.read_json("/gdrive/MyDrive/genainlp3/zno.test.jsonl", lines=True)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_eval_split = train_dataset.train_test_split(test_size=0.3, seed=42)

train_split = train_eval_split['train']
eval_split = train_eval_split['test']

print(f"Train size: {len(train_split)}, Eval size: {len(eval_split)}")

Train size: 2144, Eval size: 919


In [8]:
model_id = "unsloth/Qwen2.5-32B-bnb-4bit"

In [9]:
def stream(model, tokenizer, user_input, instruction, template, max_new_tokens=125, generation_params={}):
    runtimeFlag = "cuda:0"
    system_prompt = ''
    prompt = template.format(instruction=instruction, user_input=user_input)
    print("Input:", prompt)
    print("Output:")
    inputs = tokenizer(prompt, return_tensors="pt").to(runtimeFlag)
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=max_new_tokens, **generation_params)


In [10]:
use_flash_attn = True
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map={"": 0},
    trust_remote_code=True,
    attn_implementation="flash_attention_2" if use_flash_attn else "eager"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors.index.json:   0%|          | 0.00/280k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/4.32G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [11]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 5120)
    (layers): ModuleList(
      (0-63): 64 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=True)
          (k_proj): Linear4bit(in_features=5120, out_features=1024, bias=True)
          (v_proj): Linear4bit(in_features=5120, out_features=1024, bias=True)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=27648, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=27648, bias=False)
          (down_proj): Linear4bit(in_features=27648, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((5120,), eps=1e-05)
        (post_attention_layernorm): Qwen2RMSNorm((5120,), eps=1e-05)
      )
    )
    (norm): Qwen2RMSNorm((5120,)

In [12]:
model.enable_input_require_grads()
model.dtype

torch.float16

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.bos_token_id = 1
model.generation_config.pad_token_id = tokenizer.pad_token_id


tokenizer_config.json:   0%|          | 0.00/4.87k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

In [14]:
train_dataset[0]

{'question': 'Позначте рядок, у якому в усіх словах потрібно писати літеру *и*:',
 'answers': [{'marker': 'А',
   'text': 'бад..лина, благоч..стивий, кр..хкий, ж..виця;'},
  {'marker': 'Б', 'text': 'вар..во, меж..річчя, вич..пурений, кр..шталь;'},
  {'marker': 'В', 'text': 'п’ят..річка, заруч..ни, нев..димка, обітн..ця;'},
  {'marker': 'Г', 'text': 'зач..нати, виконав..ця, знів..чити, вел..чина;'},
  {'marker': 'Д', 'text': 'нож..чок, печ..во, викор..нити, оз..ратися.'}],
 'correct_answers': ['В'],
 'subject': 'ukrainian-language-and-literature'}

In [15]:
train_dataset.features

{'question': Value(dtype='string', id=None),
 'answers': [{'marker': Value(dtype='string', id=None),
   'text': Value(dtype='string', id=None)}],
 'correct_answers': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'subject': Value(dtype='string', id=None)}

In [16]:
TRAIN_TEMPLATE = (
    "<|im_start|>system\Ти є розумним асистентом що дає відповіді на тести українською. Відповіді повинні бути зазначені у форматі одного варіанту серед наданих: А, Б, В, Г або ж Д. Твоя відповідь має включати лише варіант відповіді на запитання з тесту.<|im_end|>\n"
    "<|im_start|>user\n{question}\n{answers}<|im_end|>\n"
    "<|im_start|>assistant\n{correct_answers}<|im_end|>\n"
)
INFERENCE_TEMPLATE = (
    "<|im_start|>system\Ти є розумним асистентом що дає відповіді на тести українською. Відповіді повинні бути зазначені у форматі одного варіанту серед наданих: А, Б, В, Г або ж Д. Твоя відповідь має включати лише варіант відповіді на запитання з тесту.<|im_end|>\n"
    "<|im_start|>user\n{question}\n{answers}<|im_end|>\n"
    "<|im_start|>assistant\n"
)


def generate_prompt(record):
    prompt = TRAIN_TEMPLATE.format(
        question=record['question'],
        answers=record['answers'],
        correct_answers=record['correct_answers']
    )
    return {'text': prompt}

train_split = train_split.map(generate_prompt)
train_split[0]


Map:   0%|          | 0/2144 [00:00<?, ? examples/s]

{'question': '**Прочитайте текст і виконайте завдання**\n\n\n(1) Тисячі дітей, *не/збайдужілих* до літератури, знають Івана Малковича – доброго дядечка-казкаря. (2) Після закінчення університету, незважаючи на різноманітні **пропозиції**, юнак, ще *не/відомий* нікому, починає вчителювати в сільській школі під Києвом. (3) Хто міг уявити тоді, що саме цей хлопець стане […] найкращого в Україні дитячого **видавництва** і, якщо треба буде, не залишатиме **друкарні** по дві доби? (4) Створені тут **книги** «Снігова королева», «Абетка», «Аліса\xa0в Дивокраї», «Тореадори з Васюківки», «Вінні-Пух» – справжні *не/перевершені* ніким витвори поліграфічного мистецтва. (5) В одному з *не/давніх* рейтингів «50 найкращих сучасних поетів» його прізвище посідає п’яте місце. (6) Вражає й такий рекорд: «А-БА-БА-ГА-ЛА-МА-ГА» за двадцять років випустила понад п’ять мільйонів книжок.\n\n\nРазом у тексті треба писати сполуку слів',
 'answers': [{'marker': 'А', 'text': 'не/збайдужілих'},
  {'marker': 'Б', 'te

In [17]:
eval_split = eval_split.map(generate_prompt)
eval_split[0]

Map:   0%|          | 0/919 [00:00<?, ? examples/s]

{'question': '**Прочитайте текст і виконайте завдання**\n\n\n(1) Тисячі дітей, *не/збайдужілих* до літератури, знають Івана Малковича – доброго дядечка-казкаря. (2) Після закінчення університету, незважаючи на різноманітні **пропозиції**, юнак, ще *не/відомий* нікому, починає вчителювати в сільській школі під Києвом. (3) Хто міг уявити тоді, що саме цей хлопець стане […] найкращого в Україні дитячого **видавництва** і, якщо треба буде, не залишатиме **друкарні** по дві доби? (4) Створені тут **книги** «Снігова королева», «Абетка», «Аліса\xa0в Дивокраї», «Тореадори з Васюківки», «Вінні-Пух» – справжні *не/перевершені* ніким витвори поліграфічного мистецтва. (5) В одному з *не/давніх* рейтингів «50 найкращих сучасних поетів» його прізвище посідає п’яте місце. (6) Вражає й такий рекорд: «А-БА-БА-ГА-ЛА-МА-ГА» за двадцять років випустила понад п’ять мільйонів книжок.\n\n\nЗамість пропуску в третьому реченні можуть бути всі слова, ОКРІМ',
 'answers': [{'marker': 'А', 'text': 'засновником'},


In [19]:
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha=64,
    lora_dropout=0.1,
    r=32,
    bias="none",
    task_type="SEQ_2_SEQ_LM",
    target_modules="all-linear"
)

In [20]:
from peft import get_peft_model

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 268,435,456 || all params: 33,032,311,808 || trainable%: 0.8126


In [21]:
import gc
torch.cuda.empty_cache()
gc.collect()

72

In [23]:
from trl import SFTConfig, SFTTrainer

output_dir = "./results"
run_name = "testrun"

training_arguments = SFTConfig(
    dataset_text_field="text",
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    optim="paged_adamw_8bit",
    save_steps=40,
    save_strategy="steps",
    eval_steps=40,
    eval_strategy="steps",
    do_eval=True,
    logging_steps=20,
    warmup_ratio=0.1,
    learning_rate=2.5e-4,
    lr_scheduler_type="linear",
    logging_dir="./logs",
    report_to="wandb",
    max_seq_length=256,
    run_name=f"{run_name}-(datetime.now().strftime('%Y-%m-%d-%H-%M'))"
)

tokenizer.padding_side = "right"

trainer = SFTTrainer(
    model=model,
    train_dataset=train_split,
    eval_dataset=eval_split,
    peft_config=peft_config,
    processing_class=tokenizer,  # Updated from tokenizer
    args=training_arguments
)

Map:   0%|          | 0/2144 [00:00<?, ? examples/s]

Map:   0%|          | 0/919 [00:00<?, ? examples/s]

In [24]:
trainer.train()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss
200,0.471700,0.584432
400,0.399900,0.659262


TrainOutput(global_step=402, training_loss=0.5462578720417782, metrics={'train_runtime': 5233.3993, 'train_samples_per_second': 1.229, 'train_steps_per_second': 0.077, 'total_flos': 3.166124284836864e+17, 'train_loss': 0.5462578720417782, 'epoch': 3.0})

In [25]:
ft_model_id = "qwen-32B-qlora-genai3-zno"

trainer.save_model(ft_model_id)
model.save_pretrained(ft_model_id, safe_serialization=False)

In [28]:
login("REDACTED")

model.push_to_hub(ft_model_id)
tokenizer.push_to_hub(ft_model_id)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


adapter_model.safetensors:   0%|          | 0.00/1.07G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/reennon/qwen-32B-qlora-genai3-zno/commit/9000e2b7e41048e773f8d4c478a1cb02afa57dc0', commit_message='Upload tokenizer', commit_description='', oid='9000e2b7e41048e773f8d4c478a1cb02afa57dc0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/reennon/qwen-32B-qlora-genai3-zno', endpoint='https://huggingface.co', repo_type='model', repo_id='reennon/qwen-32B-qlora-genai3-zno'), pr_revision=None, pr_num=None)

In [27]:
def generate_answers(record):
    prompt = INFERENCE_TEMPLATE.format(
        question=record['question'],
        answers=record['answers']
    )

    tokenized_prompt = tokenizer(prompt, return_tensors="pt")

    device = model.device
    input_ids = tokenized_prompt['input_ids'].to(device)
    attention_mask = tokenized_prompt.get('attention_mask', None)
    if attention_mask is not None:
        attention_mask = attention_mask.to(device)

    generated = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=20,
        temperature=0.02
    )

    answer = tokenizer.decode(generated[0], skip_special_tokens=True).strip()
    return {'predicted_answer': answer}

test_dataset = test_dataset.map(generate_answers)
df = test_dataset.to_pandas()
df[['id', 'predicted_answer']].to_csv('/gdrive/MyDrive/genainlp3/submission.csv', index=False)

print("Submission file 'submission.csv' created successfully.")

Map:   0%|          | 0/751 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.02` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Submission file 'submission.csv' created successfully.
